In [3]:
import numpy as np
import matplotlib.pyplot as plt

First let's define a neural network.  We'll just choose the weights and biases randomly for now

In [4]:
# Set seed so we always get the same random numbers
np.random.seed(0)

# Number of hidden layers
K = 5
# Number of neurons per layer
D = 6
# Input layer
D_i = 1
# Output layer
D_o = 1

# Make empty lists
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)

# Create input and output layers
all_weights[0] = np.random.normal(size=(D, D_i))
all_weights[-1] = np.random.normal(size=(D_o, D))
all_biases[0] = np.random.normal(size =(D,1))
all_biases[-1]= np.random.normal(size =(D_o,1))

# Create intermediate layers
for layer in range(1,K):
  all_weights[layer] = np.random.normal(size=(D, D))
  all_biases[layer] = np.random.normal(size=(D, 1))

In [5]:
# TODO Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
  return np.maximum(0, preactivation)


Now let's run our random network.  The weight matrices $\boldsymbol\Omega_{0\ldots K}$ are the entries of the list "all_weights" and the biases $\boldsymbol\beta_{0\ldots K}$ are the entries of the list "all_biases"

We know that we will need the preactivations $\mathbf{f}_{0\ldots K}$ and the activations $\mathbf{h}_{1\ldots K}$ for the forward pass of backpropagation, so we'll store and return these as well.


In [6]:
# TODO Define the forward pass, return the final output and all intermediate outputs
def compute_network_output(net_input, all_weights, all_biases):
  all_f = []  # pre-activations (before ReLU)
  all_h = [net_input]  # activations (after ReLU); first is input

  current_input = net_input
  for i in range(len(all_weights)):
    f = all_weights[i] @ current_input + all_biases[i]
    all_f.append(f)
    if i < len(all_weights) - 1:
      current_input = ReLU(f)
    else:
      current_input = f  # No activation function at final layer
    all_h.append(current_input)

  net_output = current_input
  return net_output, all_f, all_h


In [7]:
# Define input
net_input = np.ones((D_i,1)) * 1.2
# Compute network output
net_output, all_f, all_h = compute_network_output(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(1.907, net_output[0,0]))

True output = 1.907, Your answer = 1.907


Now let's define a loss function.  We'll just use the least squares loss function. We'll also write a function to compute dloss_doutput

In [8]:
# TODO Define the least squares loss and its derivative
def least_squares_loss(net_output, y):
  return 0.5 * np.sum((net_output - y)**2)

def d_loss_d_output(net_output, y):
  return net_output - y


In [9]:
y = np.ones((D_o,1)) * 20.0
loss = least_squares_loss(net_output, y)
print("y = %3.3f Loss = %3.3f"%(y, loss))

y = 20.000 Loss = 163.685


/tmp/ipykernel_6460/3361061389.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("y = %3.3f Loss = %3.3f"%(y, loss))


Now let's compute the derivatives of the network.  We already computed the forward pass.  Let's compute the backward pass.

In [10]:
# TODO Main backward pass routine, it returns the derivatives of the weights and biases
def backward_pass(all_weights, all_biases, all_f, all_h, y):
  K = len(all_weights) - 1  # Number of layers excluding input

  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)

  # Initialize delta as derivative of loss wrt output
  delta = d_loss_d_output(all_h[-1], y)

  for layer in reversed(range(K+1)):  # Start from last layer and go backward
    all_dl_dweights[layer] = delta @ all_h[layer].T
    all_dl_dbiases[layer] = delta

    if layer > 0:
      df_dh = (all_f[layer-1] > 0).astype(float)  # ReLU derivative
      delta = (all_weights[layer].T @ delta) * df_dh

  return all_dl_dweights, all_dl_dbiases


In [11]:
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)

In [12]:
np.set_printoptions(precision=3)
# Make space for derivatives computed by finite differences
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Let's test if we have the derivatives right using finite differences
delta_fd = 0.000001

# Test the dervatives of the bias vectors
for layer in range(K+1):
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer])
  # For every element in the bias
  for row in range(all_biases[layer].shape[0]):
    # Take copy of biases  We'll change one element each time
    all_biases_copy = [np.array(x) for x in all_biases]
    all_biases_copy[layer][row] += delta_fd
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("-----------------------------------------------")
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])
  if np.allclose(all_dl_dbiases_fd[layer],all_dl_dbiases[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

# Test the derivatives of the weights matrices
for layer in range(K+1):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # For every element in the bias
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Take copy of biases  We'll change one element each time
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("-----------------------------------------------")
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])
  if np.allclose(all_dl_dweights_fd[layer],all_dl_dweights[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

-----------------------------------------------
Bias 0, derivatives from backprop:
[[ -2.243]
 [  2.474]
 [  3.406]
 [ -1.942]
 [-12.468]
 [  0.   ]]
Bias 0, derivatives from finite differences
[[ -2.243]
 [  2.474]
 [  3.406]
 [ -1.942]
 [-12.468]
 [  0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 1, derivatives from backprop:
[[-0.   ]
 [-5.648]
 [ 0.   ]
 [ 0.   ]
 [-5.361]
 [ 0.   ]]
Bias 1, derivatives from finite differences
[[ 0.   ]
 [-5.648]
 [ 0.   ]
 [ 0.   ]
 [-5.361]
 [ 0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 2, derivatives from backprop:
[[-0.   ]
 [-0.   ]
 [ 0.469]
 [ 0.   ]
 [-4.997]
 [ 0.254]]
Bias 2, derivatives from finite differences
[[ 0.   ]
 [ 0.   ]
 [ 0.469]
 [ 0.   ]
 [-4.997]
 [ 0.254]]
Success!  Derivatives match.
-----------------------------------------------
Bias 3, derivatives from backprop:
[[-0.   ]
 [-2.4  ]
 [-0.831]
 [-0.   ]
 [ 1.697]
 [ 2.695]]
Bias 3, de